## 클래스, 데이터 로드

In [6]:
from key_extraction import keywordExtractor
from transformers import ElectraModel, ElectraTokenizerFast
import numpy as np
import pandas as pd
from typing import Union, Tuple, List, Dict
from itertools import chain, islice
import torch
import openai
from gensim.models import keyedvectors
import pickle

# load model and tokenizer
name = "monologg/koelectra-base-v3-discriminator"
model = ElectraModel.from_pretrained(name)
tokenizer = ElectraTokenizerFast.from_pretrained(name)

# load keywordExtractor
key = keywordExtractor(model,tokenizer,dir='data/preprocess/eng_han.csv')

# load food data
scraping_result = pd.read_csv('data/food_data.csv',encoding='cp949')
#scraping_result = pd.read_csv('data/food_data2.csv')
print('음식 데이터 수 : ', len(scraping_result))
print('')
scraping_result.head()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


음식 데이터 수 :  47



,food_name,food_category,food_description
0,라면,면,"라면은 아시아에서 유명한 인스턴트 면 요리로, 면과 스프로 구성됩니다. 면은 탄력이..."
1,야채김밥,김밥,김에 밥과 다양한 야채를 넣어서 만든 한국의 전통적인 간단한 김밥
2,치즈김밥,김밥,김에 밥과 치즈를 넣어서 만든 인기 있는 한국의 간단한 김밥
3,소고기김밥,김밥,"김에 밥과 소고기, 야채 등을 넣어서 만든 한국의 대표적인 김밥"
4,매콤오징어김밥,김밥,"오징어와 야채를 매콤한 양념으로 볶아 김에 싸서 만든, 한국의 대표적인 매운 김밥"


## 함수 선언

In [19]:
##################################### 따로 만든 함수 #####################################
API_KEY = 'sk-' ####### 키

# chatGPT API 사용 함수
def callChatGPT(prompt, API_KEY=API_KEY):
    
    messages = []

    #get api key
    openai.api_key = API_KEY

    messages.append({"role":"user", "content":prompt})

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    chat_response = completion.choices[0].message.content
    messages.append({"role":"assitant", "content":chat_response})

    return messages[1]["content"]

# chatGPT한테 필요한 데이터 얻는 함수
def obtain_data(menu_name):

    #chat_res_cat = callChatGPT(menu_name + " 는 [밥], [국], [면], [분식] 중에 뭐야")
    #chat_res_cat = chat_res_cat[chat_res_cat.find('[')+1:chat_res_cat.find(']')] # GPT 답변 : 메뉴 카테고리

    #chat_res_des = callChatGPT(menu_name + "에 대한 간단한 설명") # GPT 답변 : 메뉴 설명

    menu_name = "라면"
    chat_res_cat = "면"
    chat_res_des = '라면은 아시아에서 유명한 인스턴트 면 요리로, 면과 스프로 구성됩니다. 면은 탄력이 있고 쫄깃하며, 다양한 모양과 두께로 제작됩니다. 스프는 라면의 맛을 결정짓는 중요한 재료로, 다양한 맛과 종류가 있습니다. 라면은 추가 재료로 고기, 해산물, 채소, 계란 등을 넣어 풍부하고 맛있게 즐길 수 있습니다. 라면은 전 세계적으로 인기 있는 음식으로, 맛과 편리함으로 알려져 있습니다.'

    #menu_str = menu_name + " " + chat_res_cat + " " + chat_res_des
    menu_str = menu_name + " " + chat_res_des
    menu_list = menu_str.split()

    return menu_list

# 새로운 메뉴명 -> 메뉴명, 카테고리, 메뉴설명 -> 키워드 리스트
def get_keyword_list(menu_name):
    min_count = 2
    min_length = 1

    raw_data = obtain_data(menu_name) # 
    keyword_list = key._extract_keywords(raw_data)
    translated_keyword_list = key._map_english_to_korean(keyword_list)
    refined_keyword_list = key._eliminate_min_count_words(translated_keyword_list, min_count)
    result = list(filter(lambda x: len(x) >= min_length, refined_keyword_list))
    return (result)

# 인덱스 번호 -> 기존 메뉴들 키워드 리스트 가져오기 (지금은 사용 X)
def get_keyword_idx(num):
    min_count = 2
    min_length = 1

    doc = scraping_result.iloc[num]
  
    raw_data = _convert_series_to_list_in_main(doc)
    keyword_list = key._extract_keywords(raw_data)
    translated_keyword_list = key._map_english_to_korean(keyword_list)
    refined_keyword_list = key._eliminate_min_count_words(translated_keyword_list, min_count)
    result = list(filter(lambda x: len(x) >= min_length, refined_keyword_list))
    return (result)

# 기존 메뉴들의 food_name과 각 메뉴들에서 추출한 키워드 뽑아서 초기화하는 함수
def init_function():
    food_name = []
    food_keyword = []

    for i in range(len(scraping_result)):
        docs_keywords = extract_keyword_in_main(scraping_result.iloc[[i]])
        food_name.append(docs_keywords["food_name"][0])
        food_keyword.append(docs_keywords["keywords"][0])

    return [food_name, food_keyword]

# 메뉴 검색하는 함수
def search_menu(menu_name, food_name_list, food_keyword_list):
    search = get_keyword_list(menu_name) # 입력된 메뉴에서 키워드 추출
    print(search)

    """w2v_model = keyedvectors.load_word2vec_format('data/w2v2')

    # 키워드 확장 
    recommand_keyword = w2v_model.most_similar(positive=search, topn=15)
    np_recommand_keyword = np.array(list(map(lambda x: x[0], recommand_keyword)))
    print('W2V을 활용한 키워드 확장 :', np_recommand_keyword)
    print('')"""

    # 키워드와 유사한 도서 검색 

    user_point = [int(0)] * len(food_name_list)

    for search_key in search:
        for i in range(len(food_name_list)):
            if search_key in food_keyword_list[i]:
                user_point[i] = user_point[i] + int(1)


    """recommand_point = [int(0)] * len(food_name_list)

    for search_key in np_recommand_keyword:
        for i in range(len(food_name_list)):
            
            if search_key in food_keyword_list[i]:
                recommand_point[i] = recommand_point[i] + int(1)

    total_point = [int(0)] * len(user_point)
    for i in range(len(user_point)):
        total_point[i] = (user_point[i] * 3) + recommand_point[i]"""
    
    total_point = user_point

    top_k_idx = np.argsort(total_point)[::-1][:20]

    # 메뉴명 연관 점수 저장
    food_name_list = np.array(food_name_list)
    total_point = np.array(total_point)

    result  = dict(zip(food_name_list[top_k_idx], total_point[top_k_idx]))

    # 음식 정보 추출
    food_info = pd.read_csv('data/food_data.csv',encoding='cp949')
    IDX = food_info.food_name.isin(list(result.keys()))

    food_recommandation_result = food_info[["food_name", "food_category"]][IDX].sort_values(
        by="food_name", key=lambda x: x.map(result), ascending=False
    ).reset_index(drop=True)

    return list(food_recommandation_result.food_name)


##################################### 기존 함수 수정한 함수 #####################################

def extract_keyword_list_in_main(doc: pd.Series, min_count: int = 2, min_length: int = 2) -> List:

	raw_data = _convert_series_to_list_in_main(doc)
	keyword_list = key._extract_keywords(raw_data)
	translated_keyword_list = key._map_english_to_korean(keyword_list)
	refined_keyword_list = key._eliminate_min_count_words(translated_keyword_list, min_count)
	return list(filter(lambda x: len(x) >= min_length, refined_keyword_list))

def _convert_series_to_list_in_main(series: pd.Series) -> List[List[str]]:

	raw_data = list(series.values)
	return list(chain(*map(lambda x: x.split(), raw_data)))

def create_keyword_embedding_in_main(doc: pd.Series) -> torch.Tensor:

	keyword_list = extract_keyword_list_in_main(doc)
	tokenized_keyword = key.tokenize_keyword(keyword_list)
	return key._create_keyword_embedding(tokenized_keyword)

def create_doc_embedding_in_main(doc: pd.Series) -> torch.Tensor:

	stringified_doc = _convert_series_to_str_in_main(doc)
	tokenized_doc = key.tokenize_keyword(stringified_doc)
	return key._create_doc_embedding(tokenized_doc)

def _convert_series_to_str_in_main(series: pd.Series) -> str:
	return " ".join(list(series.values))

def extract_keyword_in_main(docs: pd.DataFrame) -> Dict:

	keyword_embedding = map(lambda x: create_keyword_embedding_in_main(x[1]), docs.iterrows())
	doc_embedding = map(lambda x: create_doc_embedding_in_main(x[1]), docs.iterrows())
	keyword_list = map(lambda x: extract_keyword_list_in_main(x[1]), docs.iterrows())

	co_sim_score = map(
		lambda x: key._calc_cosine_similarity(*x).flatten(),
		zip(doc_embedding, keyword_embedding),
	)
	top_n_keyword = list(
		map(lambda x: key._filter_top_n_keyword(*x), zip(keyword_list, co_sim_score))
	)

	return dict(food_name=docs["food_name"].tolist(), keywords=top_n_keyword)

## 시작 : 메뉴 입력 + 전체 알고리즘

In [20]:
menu_name = "참치김밥" ## 입력

lst = []

with open("data/food_name_data.pickle","rb") as fr:
    food_name_list = pickle.load(fr)

with open("data/food_keyword_data.pickle","rb") as fr:
    food_keyword_list = pickle.load(fr)

"""print('\n\n\n키워드에 따른 상위 20개 음식 추천 결과\n')
print(search_menu(menu_name, food_name_list, food_keyword_list))"""

if menu_name in food_name_list:
    print("일치하는 메뉴가 있습니다.")
    lst.append(menu_name)

else :
    lst.append(search_menu(menu_name, food_name_list, food_keyword_list))

if len(lst) == 0:
    print("해당 메뉴가 없습니다.")
else:
    print(lst)




키워드에 따른 상위 20개 음식 추천 결과

['라면', '면', '스프', '다양', '맛', '재료']
['라면', '라면', '떡라면', '콩나물라면', '만두라면', '어린이김밥', '된장찌개', '돼지김치찌개', '순두부찌개', '참치김치찌개', '매콤땡+A1초김밥', '꽁치김치찌개', '육개장', '뚝배기불고기', '김치볶음밥', '카레덮밥', '참치볶음밥', '새우날치알김밥', '참치와사비김밥', '참치마요김밥', '고등어김치찌개']


'if menu_name in food_name:\n    print("일치하는 메뉴가 있습니다.")\n    lst.append(menu_name)\n\nelse :\n    lst = search_menu(menu_name, food_name, food_keyword)\n\nif len(lst) == 0:\n    print("해당 메뉴가 없습니다.")\nelse:\n    print(lst)'

## 키워드 추출 과정

1. 데이터 전처리

In [16]:
min_count = 2
min_length = 1
doc = scraping_result.iloc[0]

#print(f'음식 정보 \n \n {doc} \n \n')

## raw_data = key._convert_series_to_list(doc)
raw_data = _convert_series_to_list_in_main(doc)

print(f'\n1. 음식 정보를 list로 통합 -> {len(raw_data)} 개 단어')
print(f'{raw_data[:10]}.... \n')

keyword_list = key._extract_keywords(raw_data)
print(f'2. 형태소 분석기를 활용해 명사만을 추출 -> {len(keyword_list)} 개 단어')
print(f'{keyword_list[:10]}.... \n')

translated_keyword_list = key._map_english_to_korean(keyword_list)
print(f'3. 영단어를 한글로 변환(ex python -> 파이썬) -> {len(translated_keyword_list)} 개 단어')
print(f'{translated_keyword_list[:10]}.... \n')

refined_keyword_list = key._eliminate_min_count_words(translated_keyword_list, min_count)
print(f'4. 최소 2번이상 반복 사용되는 단어만 추출 -> {len(refined_keyword_list)} 개 단어')
print(f'{refined_keyword_list[:10]}.... \n')

result = list(filter(lambda x: len(x) >= min_length, refined_keyword_list))
print(f'5. 단어 길이가 최소 한개 이상인 단어만 추출 -> {len(result)} 개 단어')
print(f'{result[:10]}.... \n')


1. 음식 정보를 list로 통합 -> 53 개 단어
['라면', '면', '라면은', '아시아에서', '유명한', '인스턴트', '면', '요리로,', '면과', '스프로'].... 

2. 형태소 분석기를 활용해 명사만을 추출 -> 37 개 단어
['라면', '면', '라면', '아시아', '유명', '인스턴트', '요리', '면', '스프', '구성'].... 

3. 영단어를 한글로 변환(ex python -> 파이썬) -> 37 개 단어
['라면', '면', '라면', '아시아', '유명', '인스턴트', '요리', '면', '스프', '구성'].... 

4. 최소 2번이상 반복 사용되는 단어만 추출 -> 6 개 단어
['라면', '면', '스프', '다양', '맛', '재료'].... 

5. 단어 길이가 최소 한개 이상인 단어만 추출 -> 6 개 단어
['라면', '면', '스프', '다양', '맛', '재료'].... 



2. 키워드 뽑기 + 임베딩 생성

In [17]:
from pprint import pprint

doc = scraping_result.iloc[0]

print(f'-- 메뉴명 -- \n {doc.food_name} \n \n')

keyword_list = extract_keyword_list_in_main(doc)
print(f'음식에 대한 키워드 후보 : {len(result)} 개 단어')
print(f'{result[:10]}.... \n \n')

keyword_embedding = create_keyword_embedding_in_main(doc)
doc_embedding = create_doc_embedding_in_main(doc)

-- 메뉴명 -- 
 라면 
 

음식에 대한 키워드 후보 : 6 개 단어
['라면', '면', '스프', '다양', '맛', '재료'].... 
 

tensor([[-0.2670,  0.1418, -0.5752,  ...,  0.3847,  0.5255, -0.1250],
        [-0.4726,  0.4308, -0.7058,  ...,  0.4029,  0.3145, -0.0826],
        [-0.3596,  0.3184, -0.4528,  ...,  0.1221,  0.1495,  0.0995],
        [-0.2679,  0.4361, -0.5833,  ...,  0.5173,  0.0931, -0.2986]])
라면 면 라면은 아시아에서 유명한 인스턴트 면 요리로, 면과 스프로 구성됩니다. 면은 탄력이 있고 쫄깃하며, 다양한 모양과 두께로 제작됩니다. 스프는 라면의 맛을 결정짓는 중요한 재료로, 다양한 맛과 종류가 있습니다. 라면은 추가 재료로 고기, 해산물, 채소, 계란 등을 넣어 풍부하고 맛있게 즐길 수 있습니다. 라면은 전 세계적으로 인기 있는 음식으로, 맛과 편리함으로 알려져 있습니다.
{'input_ids': tensor([[    2,  6644,  2672,  6644,  4112,  7198,  4073,  4129,  6963,  4283,
         26043,  2672,  7747,  4239,    16,  2672,  4047, 10824,  4239,  6537,
          4608,  6216,    18,  8860, 10737,  4007,  3249,  4219, 19561,  4279,
          4815,    16,  6417,  4283,  7415,  4047, 14609,  4239,  6815,  4608,
          6216,    18, 10824,  4034,  6644,  4234,  2637,  4292,  6393,  4637,
       

3. 코사인 유사도순으로 정렬

In [13]:
co_sim_score = key._calc_cosine_similarity(doc_embedding, keyword_embedding).flatten()

keyword = dict(zip(keyword_list, co_sim_score))

sorted_keyword = sorted(keyword.items(), key=lambda k: k[1], reverse=True)

print(f'-- 키워드 추출 결과(20개 요약)--')
pprint(sorted_keyword[:20])

-- 키워드 추출 결과(20개 요약)--
[('라면', 0.85003704), ('다양', 0.8180337), ('재료', 0.8042269), ('스프', 0.7779288)]


In [18]:
# extract keywords
docs_keywords = extract_keyword_in_main(scraping_result.iloc[[0]])

# result
result = pd.DataFrame(docs_keywords)

print('키워드 추출 예시\n')
print('메뉴명 : ', scraping_result.iloc[0].food_name)
pd.DataFrame(result.keywords.values[0])

라면 면 라면은 아시아에서 유명한 인스턴트 면 요리로, 면과 스프로 구성됩니다. 면은 탄력이 있고 쫄깃하며, 다양한 모양과 두께로 제작됩니다. 스프는 라면의 맛을 결정짓는 중요한 재료로, 다양한 맛과 종류가 있습니다. 라면은 추가 재료로 고기, 해산물, 채소, 계란 등을 넣어 풍부하고 맛있게 즐길 수 있습니다. 라면은 전 세계적으로 인기 있는 음식으로, 맛과 편리함으로 알려져 있습니다.
{'input_ids': tensor([[    2,  6644,  2672,  6644,  4112,  7198,  4073,  4129,  6963,  4283,
         26043,  2672,  7747,  4239,    16,  2672,  4047, 10824,  4239,  6537,
          4608,  6216,    18,  8860, 10737,  4007,  3249,  4219, 19561,  4279,
          4815,    16,  6417,  4283,  7415,  4047, 14609,  4239,  6815,  4608,
          6216,    18, 10824,  4034,  6644,  4234,  2637,  4292,  6393,  4637,
          4034,  6397,  4283,  8282,  4239,    16,  6417,  4283, 16970,  7890,
          4070,  3249,  4576,  6216,    18,  6644,  4112,  6565,  8282,  4239,
          8097,    16, 19733,    16, 11109,    16, 11581,  2446,  4292,  2278,
          4025,  9005,  4279,  4219,  7461,  4325,  9019,  2967,  3249,  4576,
          6216,    18,  6644,  4112,  3280,  6287,  4199,

,0
0,라면
1,다양
2,재료
3,스프
